# Calculate D-statistic using frequency data
1. Making population file, listing sets of four populations to calculate D for. 
2. R script for calculating D, modified from original script sent by Anders. 
3. Python function to turn resulting .mat file into matrix of Z-scores (Zmat) and D values (Dmat)

In [129]:
%%writefile /mnt/solexa/mel_yang/BGI/makepopfile_BGI_BGI_p3.py
## python makepopfile.py p3popname

import sys
import projmodules as pm
pd="/mnt/solexa/mel_yang/BGI/"
anc=["Mbuti"]
p3s=[sys.argv[1]] #,"Han","Dai","Daur","Boisman_MN","BoshanChineseNeolithic","LiangDaoChineseNeolithic"]

minofile=open(pd+"BGI/Minority_nind.txt",'r')
minopops=[line.split()[0] for line in minofile]
minofile.close()

hanfile=open(pd+"BGI/OnlyHan_nind.txt",'r')
bgipops=[line.split()[0] for line in hanfile]
hanfile.close()
ranpops=[i+"_N" for i in bgipops]
hanpops=[i+"_H" for i in bgipops]

p1and2 = [ (i,j) for i in hanpops+ranpops+minopops for j in hanpops+ranpops+minopops ]


pm.Dpopfile(p1and2,p3s,anc,pd+"HO6_BGI","BGI_BGI_%s_Mbuti" % p3s[0],f4mode=False)

Overwriting /mnt/solexa/mel_yang/BGI/makepopfile_BGI_BGI_p3.py


In [128]:
%%writefile /mnt/solexa/mel_yang/BGI/makepopfile_EAS_EAS_p3.py
## python makepopfile.py p3popname

import sys
import projmodules as pm
pd="/mnt/solexa/mel_yang/BGI/"
anc=["Mbuti"]
p3s=[sys.argv[1]] #,"Han","Dai","Daur","Boisman_MN","BoshanChineseNeolithic","LiangDaoChineseNeolithic"]

eas=["Ami","Atayal","Dai","Daur","Han","Hezhen","Japanese","Korean","Kinh",
     "Lahu","Miao","Naxi","Oroqen","She","Tujia","Yi"]

ancasn=["Tianyuan",'Longlin','Longlin_com','Xinyi','Xinyi_other','Daxi',
        'LiangDaoChineseNeolithic','Boisman_MN','BoshanChineseNeolithic',
        'Bbdong','Linzi','HDYM1','HQSDW','Zongri','Pukagongma']

p1and2 = [ (i,j) for i in eas+ancasn for j in eas+ancasn ]


pm.Dpopfile(p1and2,p3s,anc,pd+"UPA8_BGI","EAS_EAS_%s_Mbuti" % p3s[0],f4mode=False)

Overwriting /mnt/solexa/mel_yang/BGI/makepopfile_EAS_EAS_p3.py


In [1]:
import projmodules as pm
pd="/mnt/solexa/mel_yang/BGI/"
anc=["Mbuti"]
p1s=["Xinyi"] #,"Han","Dai","Daur","Boisman_MN","BoshanChineseNeolithic","LiangDaoChineseNeolithic"]
p1s=["Dai"]

minofile=open(pd+"BGI/Minority_nind.txt",'r')
minopops=[line.split()[0] for line in minofile]
minofile.close()

hanfile=open(pd+"BGI/OnlyHan_nind.txt",'r')
bgipops=[line.split()[0] for line in hanfile]
hanfile.close()
ranpops=[i+"_N" for i in bgipops]
hanpops=[i+"_H" for i in bgipops]

p3s=hanpops+ranpops+minopops
p1and2 = [ (p1s[0],j) for j in hanpops+ranpops+minopops ]


#pm.Dpopfile(p1and2,p3s,anc,pd+"UPA8_BGI","%s_BGI_BGI_Mbuti" % p1s[0],f4mode=False)
pm.Dpopfile(p1and2,p3s,anc,pd+"HO6_BGI","%s_BGI_BGI_Mbuti" % p1s[0],f4mode=False)

10000

In [30]:
import projmodules as pm
pd="/mnt/solexa/mel_yang/BGI/"
anc=["Mbuti"]
p3s=["Xinyi"] #,"Han","Dai","Daur","Boisman_MN","BoshanChineseNeolithic","LiangDaoChineseNeolithic"]

eup=["GoyetQ116-1","Vestonice16","Yana_old","Yana_old2",
     "Malta1",'AfontovaGora3',"ElMiron","Villabruna","Satsurblia","Karelia",
     "Loschbour","LaBrana1","Hungarian.KO1","Stuttgart","French"]

#p1and2 = [ (i,j) for i in ["Han","Ami","Dai"]+hanpops+ranpops+minopops for j in ["Vestonice16","Kostenki14"] ]
p1and2 = [ (i,j) for i in eup for j in eup if i!=j ]


pm.Dpopfile(p1and2,p3s,anc,pd+"UPA8_BGI","ANCEUR_ANCEUR_%s_Mbuti" % p3s[0],f4mode=False)

210

## 2. R script to calculate D-stat using frequency data

In [8]:
%%writefile /mnt/solexa/mel_yang/BGI/Dfreq.R
args <- commandArgs(trailingOnly = TRUE)

if(length(args) < 2) {
  args <- c("--help")
}
## Help section
if("--help" %in% args) {
  cat("
      Example:

      Rscript Dfreq.R  /mnt/solexa/mel_yang/BGI/ UPA8_BGI BGI_BGI_LiangDaoChineseNeolithic_Mbuti

      Rscript plotPcaFromSmartpca.R  pd fh label
      Note that fh should have an '_freqs.txt' and a '_nind.txt' file
      The label should go with the .pop file, which should be of the form pd+fh+'.D.'+label+'.pop'\n\n")
    
  q(save="no")
}

#Frequencies for 1000 SNPs for 10 indivduals
library(iotools)

pd= args[1] #"/mnt/solexa/mel_yang/BGI/"
fh=args[2] #"UPA8_BGI"
label=args[3] #"BGI_BGI_LiangDaoChineseNeolithic_Mbuti"

####### functions ################

blockJackUneven<-function(dat,theta=function(x) sum(x[,1])/sum(x[,2])){
   dat<-dat[dat[,3]>0,]
   nblocks<-dim(dat)[1]

   thetaEst<-theta(dat)
   etai<-rep(0, nblocks)
   blockSize<-dat[,3]
   blockFrac<-blockSize/sum(blockSize)

   for(i in 1:nblocks)
     etai[i]<-theta(dat[-i,])

   meanJack<-mean(etai)
   jackEst<-nblocks*thetaEst-sum((1-blockFrac)*etai)
   jackVar<-(nblocks-1)/nblocks * sum((etai-meanJack)^2)
   jackVar<-1/nblocks * sum( 1/(1/blockFrac-1) * 
(1/blockFrac*thetaEst-(1/blockFrac-1)*etai - 
nblocks*thetaEst+sum((1-blockFrac)*etai))^2)

  return(c(jackVar=jackVar,jackEst=jackEst))
}

getD<-function(x,freq,siteInfo){


  f<-freq[,x]
  #keep1<-rowSums(is.na(f))==0
  keep1<-is.na(rowSums(f))==0 ##CHANGED

   if(sum(keep1)<10){
      res<-rep(NA,8)
names(res)<-c("D","Djack","SE","Z","diff","total","Nsites","Nblocks")
      return(res)

  }
    
  f=freq[keep1,x] ##ADDED

  n<-(f[,3]-f[,4])*(f[,2]-f[,1])

  d<-(f[,3]+f[,4]-2*f[,3]*f[,4])*(f[,1]+f[,2]-2*f[,1]*f[,2])


  res<-1
  keep<-d>0
  n<-n[keep]
  d<-d[keep]
  est<-sum(n)/sum(d)
  nb<-ctapply(n,siteInfo$block[keep1][keep],sum)
  db<-ctapply(d,siteInfo$block[keep1][keep],sum)
  lb<-ctapply(d,siteInfo$block[keep1][keep],length)
  Z<-NA
  se<-NA
  estJ<-NA

  if(length(lb)>19){
      jackRes<-blockJackUneven(cbind(nb,db,lb))
      estJ<-jackRes[2]
      se<-sqrt(jackRes[1])
      Z<-est/se
  }
  res<-c(est,estJ,se,Z,sum(n),sum(d),sum(keep),length(nb))
  names(res)<-c("D","Djack","SE","Z","diff","total","Nsites","Nblocks")

  return(res)
}

#freqfile=read.table(paste(pd,"UPA8_BGI_freqs.test",sep=""))
freqfile=read.table(paste(pd,fh,".noAG.noCT_freqs.txt",sep=""))
indfile=read.table(paste(pd,fh,"_nind.txt",sep=""))
mypatts=read.table(paste(pd,fh,".D.",label,".pop",sep=""))

print ("Read all information in!")

chr=freqfile$V1
pos=freqfile$V2
freqfile=freqfile[,3:ncol(freqfile)]
freq=as.matrix(freqfile)

colnames(freq) <- indfile$V1

#chr <- rep(1:10,each=100)
#pos <- unlist(lapply(1:10,function(x) sort(as.integer(runif(100)*1e8))))

block<-paste(chr,cut(pos,seq(1,3e8,by=5e6))) # 5 MB blocks
siteInfo <- data.frame(pos=pos,chr=chr,block=block)

## get Dstat for tree (((pop1,pop2),pop5,pop6)
mylabels=c()
for (i in 1:nrow(mypatts)) {
    y=c(as.character(mypatts[i,1]),as.character(mypatts[i,2]),as.character(mypatts[i,3]),as.character(mypatts[i,4]))
    mylabels=c(mylabels,do.call(paste, c(as.list(y), sep=",")))
    #getD(c("Hanzu","Naxizu","Yaozu","Hanizu"),freq=freq,site=siteInfo)
    mymat=as.matrix(getD(y,freq=freq,site=siteInfo))
    if (i==1) {
        finmat=mymat
    }
    else{
        finmat=cbind(finmat,mymat)
    }   
}
colnames(finmat) = mylabels
finmat=t(finmat)
write.table(finmat,paste(pd,fh,".D.",label,".noAG.noCT.mat",sep=""),quote=F,sep="\t")



Overwriting /mnt/solexa/mel_yang/BGI/Dfreq.R


## 3. Python script to make Zmat and Dmat

In [2]:

def Dfreqmatrix(pd,fh,popindex1,popindex2,ZorD="Z"):
    import numpy as np
    
    newfile=open(pd+fh+".%smat.csv" % ZorD,'w')
    myset1,myset2=[],[]
    myinfo={}
    myfile=open(pd+fh+".mat",'r')
    for ind,line in enumerate(myfile):
        if ind==0: header=line.split(); continue
        x=line.split()
        pop1=x[0].split(',')[popindex1]
        pop2=x[0].split(',')[popindex2]
        if pop1 not in myset1: myset1.append(pop1)
        if pop2 not in myset2: myset2.append(pop2)
        statindex=header.index(ZorD)+1
        mystat=float(x[statindex]) if x[statindex]!="NA" else 0
        myinfo[(pop1,pop2)]=mystat
        #if mystat=="NA": stats.append(0)
        #else: stats.append(float(mystat))
    
    myary=np.zeros((len(myset1),len(myset2)))
    for ind1,p1 in enumerate(myset1):
        for ind2,p2 in enumerate(myset2):
            if p1==p2: continue
            myary[ind1,ind2]=myinfo[(p1,p2)]
    #myary=np.array(stats).reshape(len(myset1),len(myset2))
    
    newfile.write("%i/%i," % (popindex1,popindex2) +",".join(myset2)+'\n')
    for myind in range(myary.shape[0]):
        mystr=",".join(["%.3f" % i for i in myary[myind,:]])
        newfile.write(myset1[myind]+","+mystr+"\n")
    newfile.close()
    myfile.close()
    
pD="/mnt/solexa/mel_yang/BGI/"
#fh="UPA8_BGI.D.BGI_BGI_Dai_Mbuti"
fh="UPA8_BGI.D.Dai_BGI_BGI_Mbuti"
fh="UPA8_BGI.D.ANCEUR_ANCEUR_Tianyuan_Mbuti"
fh="UPA8_BGI.D.EAS_EAS_Ami_Mbuti"
Dfreqmatrix(pD,fh,0,1,'Z')

In [3]:

def Dfreqmatrix(pd,fh,popindex1,popindex2,ZorD="Z",fmt="xlsx",newname=""):
    import numpy as np
    
    if newname=="": newfilename=pd+fh+".%smat" % ZorD
    else: newfilename=pd+newname+".%smat" % ZorD
        
    if fmt=="csv": newfile=open(newfilename+".csv",'w')
    elif fmt=="xlsx": 
        import xlsxwriter as xls
        newfile=xls.Workbook(newfilename+".xlsx")
        worksheet=newfile.add_worksheet(ZorD)
        
        mynum = newfile.add_format({'num_format': '0.0','center_across':True})
        bold = newfile.add_format({'bold': True,'align':'center','valign':'vcenter'})
        colheader=newfile.add_format({'bold': True,'align':'center','valign':'vcenter','rotation':90})
        nan = newfile.add_format({'align':'center','valign':'vcenter'})
        
        ##SWITCHED FROM TY FORMATS BC DFREQ DOES OPPOSITE ABBA-BABA (not BABA-ABBA!)
        l2_5 = newfile.add_format({'center_across':True,'bold':True,
                                   'font_color':'#9C0006','bg_color':'white',
                                   'num_format': '0.0'})
        l3 = newfile.add_format({'center_across':True,'bold':True,'font_color':'#9C0006',
                                   'bg_color':'#FFC7CE','num_format': '0.0'})
        g2_5 = newfile.add_format({'center_across':True,'bold':True,'font_color':'#5B3317',
                                   'bg_color':'white','num_format': '0.0'})
        g3 = newfile.add_format({'center_across':True,'bold':True,'font_color':'#5B3317',
                                   'bg_color':'#FBEFB8','num_format': '0.0'})
        
    else: return "Pick fmt=csv or xlsx!"
    
    myset1,myset2=[],[]
    myinfo={}
    myfile=open(pd+fh+".mat",'r')
    for ind,line in enumerate(myfile):
        if ind==0: header=line.split(); continue
        x=line.split()
        mypops=x[0].split(',')
        pop1=mypops[popindex1]
        pop2=mypops[popindex2]
        mypops.remove(pop1)
        mypops.remove(pop2)
        if pop1 not in myset1: myset1.append(pop1)
        if pop2 not in myset2: myset2.append(pop2)
        statindex=header.index(ZorD)+1
        mystat=float(x[statindex]) if x[statindex] not in ["NA","Inf"] else float("nan")
        if pop1 in mypops or pop2 in mypops or pop1==pop2: myinfo[(pop1,pop2)]=float("nan")
        else: myinfo[(pop1,pop2)]=mystat
        #if mystat=="NA": stats.append(0)
        #else: stats.append(float(mystat))
    
    myary=np.zeros((len(myset1),len(myset2)))
    for ind1,p1 in enumerate(myset1):
        for ind2,p2 in enumerate(myset2):
            if p1==p2: continue
            myary[ind1,ind2]=myinfo[(p1,p2)]
    #myary=np.array(stats).reshape(len(myset1),len(myset2))
    if fmt=="csv":
        newfile.write("%i/%i," % (popindex1,popindex2) +",".join(myset2)+'\n')
        for myind in range(myary.shape[0]):
            mystr=",".join(["%.3f" % i for i in myary[myind,:]])
            newfile.write(myset1[myind]+","+mystr+"\n")
    elif fmt=="xlsx":
        worksheet.set_column(1,len(myset2)+1,4)
        row,col=0,0
        worksheet.write_string(row,col,"P%i/P%i" % (popindex1+1,popindex2+1),bold)
        for ind,i in enumerate(myset2): worksheet.write_string(row,col+ind+1,i,colheader)
        for myind in range(myary.shape[0]):
            worksheet.write_string(row+myind+1,col,myset1[myind],bold)
            for myind2,i2 in enumerate(myary[myind,:]):
                if str(i2)=="nan": 
                    worksheet.write_string(row+myind+1,col+myind2+1,"nan",nan)
                else:
                    if i2>=2.5 and i2<3: 
                        worksheet.write_number(row+myind+1,col+myind2+1,i2,g2_5)
                    elif i2>=3:
                        worksheet.write_number(row+myind+1,col+myind2+1,i2,g3)
                    elif i2<=-2.5 and i2>-3:
                        worksheet.write_number(row+myind+1,col+myind2+1,i2,l2_5)
                    elif i2<=-3:
                        worksheet.write_number(row+myind+1,col+myind2+1,i2,l3)
                    else:
                        worksheet.write_number(row+myind+1,col+myind2+1,i2,mynum)

    else: return "Pick fmt=csv or xlsx!"
    newfile.close()
    myfile.close()
    
myset=["Dai","Daur","Ami","Han","LiangDaoChineseNeolithic","Naxi","Yi","Xibo","Miao","Tujia","She","Tu"]
myset=["Dai","Naxi","Yi","Xibo","Miao","Tujia","She","Tu","Lahu","Dong","Li","Gelao","Qiang_Horpa","Xibo","Zhuang"]
pD="/mnt/solexa/mel_yang/BGI/"
#fh="UPA8_BGI.D.BGI_BGI_Dai_Mbuti"
#fh="UPA8_BGI.D.Dai_BGI_BGI_Mbuti"
#for i in myset:
#    fh="UPA8_BGI.D.BGI_BGI_%s_Mbuti" % i
#    fh="HO6_BGI.D.BGI_BGI_%s_Mbuti" % i
#    newfh=""
#    if i =="LiangDaoChineseNeolithic": newfh="UPA8_BGI.D.BGI_BGI_%s_Mbuti" % "LD"
#    Dfreqmatrix(pD,fh,0,1,'Z','xlsx',newfh)
Dfreqmatrix(pD,"UPA8_BGI.D.ANCEUR_ANCEUR_Tianyuan_Mbuti",0,1,'Z','xlsx',"")

In [7]:
myset=["Dai","Daur","Ami","Han","LiangDaoChineseNeolithic","Longlin","Tianyuan"]
for i in myset: 
    Dfreqmatrix(pD,"UPA8_BGI.D.EAS_EAS_%s_Mbuti" % i,0,1,'Z','xlsx',"")